<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/library/tensorflow/%E5%9C%A8colab%E9%96%8B%E5%A7%8B%E5%AD%B8%E8%B5%B7tf/00/00.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

使用 Tensorflow v2  
導入 Tensorflow 與 Numpy

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.2.0-rc2
1.18.2


<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.08665041], dtype=float32)>

In [0]:
def generate_data(data_num, answer):
    x = tf.random.normal((data_num,1))
    y = tf.multiply(x, answer)
    return [x,y]

@tf.function
def model(w,x):
    return tf.multiply(w, x)

def train(model, w, x, y, epoch, lr=0.001):
    optimizer = tf.optimizers.Adamax(lr)
    for i in range(epoch):
        with tf.GradientTape() as tape:
            y_pred = model(w, x)
            loss = tf.losses.mean_squared_error(y_pred=y_pred, y_true=y)
        grads = tape.gradient(loss, [w])
        # print(grads)
        optimizer.apply_gradients(zip(grads, [w]))


In [46]:
w = tf.Variable(tf.random.normal([1]))
[x, y] = generate_data(10000, 20)

print(tf.reduce_mean(tf.losses.mean_squared_error(y_pred=model(w,x), y_true=y)))

for i in range(5):
    train(model, w, x, y, 100, 0.1)
    print(tf.reduce_mean(tf.losses.mean_squared_error(y_pred=model(w,x), y_true=y)))

print(w)

tf.Tensor(377.90637, shape=(), dtype=float32)
tf.Tensor(120.86247, shape=(), dtype=float32)
tf.Tensor(14.766934, shape=(), dtype=float32)
tf.Tensor(0.0033662564, shape=(), dtype=float32)
tf.Tensor(7.214745e-10, shape=(), dtype=float32)
tf.Tensor(7.5189206e-08, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([19.999727], dtype=float32)>
